# Тарау 1: Мәтіндерді өңдеу
# Раздел 1: Обработка текстовых данных

1. Бұл блокнотта қолданылатын пакеттер:
2. Библиотеки, которые используются в этом блокноте:

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

- Бұл тарауда  деректерді өңдеу және бөлшектерге бөлу қарастырылады
- В этой главе мы рассмотрим обработку текстовых данных и разделение их на части.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="1000px">

## 2.1 Cөз эмбедингтерін (векторлық көріністер) түсіндіру
## 2.1 Понимание эмбеддингов (векторные представления) слов

Эмбеддингтердің көптеген түрлері бар; бұл жұмыста біз мәтіндік эмбеддингтерге назар аударамыз.

Существует множество видов эмбеддингов; в этой работе мы сосредоточимся на текстовых эмбеддингах.



<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="1000px">

- Ауқымды Тіл Модельдері (ATM) жоғары өлшемді кеңістіктердегі (яғни, мыңдаған өлшемдері бар) эмбеддингтермен жұмыс істейді.
- Языковые модели работают с эмбеддингами в высокоразмерных пространствах (т. е. с тысячами измерений).
***
- Біз мұндай жоғары өлшемді кеңістіктерді көрнекі түрде елестете алмайтындықтан (өйткені адамдар 1, 2 немесе 3 өлшеммен ойлайды),     
төмендегі суретте 2 өлшемді эмбеддинг кеңістігі көрсетілген.
- Поскольку мы не можем визуализировать такие многомерные пространства (ведь люди мыслят в 1, 2 или 3 измерениях),    
на рисунке ниже показано двумерное пространство эмбеддингов.


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="1000px">

## 2.2 Текст токендеу
## 2.2 Токенизация текста

- Бұл бөлімде біз мәтінді токендейміз, яғни оны жеке сөздер мен тыныс белгілері сияқты кішірек бірліктерге бөлеміз.
- В этом разделе мы токенизируем текст, то есть разделяем его на более мелкие единицы, такие как отдельные слова и знаки препинания.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="1000px">

- Бастапқы мәтін ретінде Герберт Уэлс жазушының "Time Machine" шығармасын қолданамыз.
- В качестве исходного текста мы используем произведение Герберта Уэлса "Time Machine".

In [ ]:
import os
import urllib.request

if not os.path.exists("time_machine.txt"):
    url = ("https://raw.githubusercontent.com/Azamat0315277/LLM_from_scratch/refs/heads/main/ch01/time_machine.txt")
    file_path = "time_machine.txt"
    urllib.request.urlretrieve(url, file_path)

In [ ]:
with open("time_machine.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[20:200])

- Мақсат — осы мәтінді АТМ үшін токендеу және одан эмбединг жасау.  
- Кейінірек жоғарыдағы мәтінге қолдану үшін, қарапайым үлгі мәтін негізінде қарапайым токендегіш әзірлейміз.   
- Төмендегі тұрақты өрнек (Regular Expression)  сөздерді бос орындар бойынша бөледі.    

 <br>
 
- Цель — токенизировать текст и создать эмбеддинги для LLM.  
- Давайте разработаем простой токенизатор на основе простого примера текста, который мы затем сможем применить к тексту выше.
- Следующее регулярное выражение (Regular Expression) будет выполнять разделение по пробельным символам.

In [ ]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

- Бізге мәтінді тек бос орындар бойынша ғана емес, үтірлер мен нүктелер бойынша да бөлу керек, сондықтан осыған сәйкес тұрақты өрнекті өзгертейік.
- Нам необходимо выполнять разделение не только по пробельным символам, но и по запятым и точкам, поэтому давайте изменим регулярное выражение чтобы оно учитывало и это.

In [ ]:
result = re.split(r'([,.]|\s)', text)

print(result)

- Көріп тұрғанымыздай, бұл сөз арасындағы бос орындарды көбейтті, сондықтан оларды да алып тастайық.
- Как мы видим, это создаёт пустые строки — давайте и их удалим.

In [ ]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

- Бұл әжептәуір жақсы, бірақ нүктелер, сұрақ белгілері және т.б. сияқты тыныс белгілерінің басқа түрлерін де ескерейік.
- Выглядит неплохо, но давайте также обработаем и другие знаки препинания, такие как точки, вопросительные знаки и так далее.

In [ ]:
text = "Hello, world—. Is this-- a test?"

result = re.split(r'([,.:;?_!"()—\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

- Нәтиже жақсы, енді біз осы токендеу әдісін бастапқы мәтінге қолдануға дайынбыз.
- Очень неплохо, и теперь мы готовы применить эту токенизацию к исходному тексту.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="1000px">

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()—*’“$%\']|--|——|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

- Токендердің жалпы санын есептейік.
- Давайте посчитаем общее количество токенов.

In [ ]:
print(len(preprocessed))

## 2.3 Токендерді токен ID-ларына айналдыру
## 2.3 Преобразование токенов в идентификаторы (ID)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="1000px">

- Енді осы токендерден барлық бірегей токендерді қамтитын сөздік құра аламыз.
- Теперь из этих токенов можно составить словарь, состоящий из всех уникальных токенов.

In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

- Сөздік құру
- Составляем словарь

In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- Сөздіктегі алғашқы 50 сөзді көрсету
- Ниже приведены первые 50 элементов этого словаря:

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 100:
       break

## Link:
* Tokenizer visualizer: https://tiktokenizer.vercel.app

- Төменде шағын сөздікті пайдаланып, қысқа үлгі мәтінді токендеу үдерісі көрсетілген:
- Ниже показана токенизация небольшого образца текста с использованием маленького словаря:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="1000px">

- Енді барлығын токендеу `Класына` біріктіреміз:
- Теперь соберём всё вместе в `Kласс` токенизатора

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        # Preprocess the text
        preprocessed = re.split(r'([,.:;?_!"()—*’“$%\']|--|——|\s)', text)
        # Remove empty strings
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        # Convert strings to IDs
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?_!"()—*’“$%\'])', r'\1', text)
        return text

- `encode` функциясы **мәтінді** **токен ID-ларына** айналдырады.
- `decode` функциясы **токен ID-ларын** **кері мәтінге** айналдырады.

<br>

- Функция `encode` преобразует **текст** в **идентификаторы токенов (ID)**.
- Функция `decode` преобразует **идентификаторы токенов (ID)** обратно в **текст**.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="1000px">

- Біз токендеу құралын пайдаланып, **мәтіндерді** **бүтін сандарға** кодтай (яғни, токендей) аламыз.
- Содан кейін бұл **бүтін сандарды** АТМ үшін кіріс деректері ретінде **эмбеддинг жасауға** болады.

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """Well, I do not mind telling you I have been at work upon this geometry
          of Four Dimensions for some time"""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

- Бүтін сандарды кері мәтінге декодтауға болады. 
- Мы можем декодировать целые числа обратно в текст.

In [ ]:
tokenizer.decode(ids)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

## 2.4 **Арнайы контекст** бар токендерін қосу 
## 2.4 Добавление **специальных контекстных** токенов

- **Бейтаныс сөздерді** және **мәтіннің соңын** белгілеу үшін бірнеше «арнайы» токендер қосқан пайдалы.
- Полезно добавить несколько «специальных» токенов для обозначения **неизвестных слов** и **конца текста**.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="1000px">

- Кейбір токенизаторлар АТМ-ге қосымша контекст беру үшін арнайы токендерді пайдаланады.  
Осындай арнайы токендердің кейбірі:  
    - [BOS] (тізбектің басы) мәтіннің басын белгілейді.
    - [EOS] (тізбектің соңы) мәтіннің қай жерде аяқталатынын белгілейді (бұл әдетте бір-бірімен байланыссыз бірнеше мәтінді, мысалы, екі түрлі Уикипедия мақаласын немесе екі түрлі кітапты біріктіру үшін қолданылады).
    - [PAD] (толтыру) егер АТМ-ді 1-ден үлкен топтама өлшемімен оқытсақ қолданылады (біз әртүрлі ұзындықтағы бірнеше мәтінді қосуымыз мүмкін; толтыру токенімен барлық мәтіндердің ұзындығы бірдей болуы үшін қысқа мәтіндерді ең ұзынына дейін толтырамыз).
    - [UNK] сөздікке кірмеген сөздерді белгілеу үшін қолданылады.
- Айта кететін жайт, `GPT-2` жоғарыда аталған токендердің ешқайсысын қажет етпейді, бірақ күрделілікті азайту үшін тек  `<|endoftext|>` токенін пайдаланады.
`<|endoftext|>` жоғарыда аталған [EOS] токеніне ұқсас.  
- GPT сондай-ақ `<|endoftext|>` токенін толтыру үшін де пайдаланады (себебі біз топтамалық енгізулермен оқыту кезінде әдетте масканы қолданамыз, сондықтан толтырылған токендерге назар аудармаймыз, сәйкесінше ол токендердің қандай болғаны маңызды емес).
- GPT-2 сөздіктен тыс сөздер үшін <UNK> токенін пайдаланбайды; оның орнына GPT-2 байт жұбымен кодтау (BPE) токенизаторын қолданады, ол сөздерді біз кейінгі бөлімде талқылайтын сөз бөліктеріне бөледі.



- Некоторые токенизаторы используют специальные токены, чтобы предоставить LLM дополнительный контекст.   
Вот некоторые из этих специальных токенов:
    - [BOS] (начало последовательности) отмечает начало текста.
    - [EOS] (конец последовательности) отмечает место окончания текста (обычно используется для объединения нескольких несвязанных текстов, например, двух разных статей из Википедии или двух разных книг и т. д.).
    - [PAD] (заполнение/паддинг) используется, если мы обучаем LLM с размером батча больше 1 (мы можем включать несколько текстов разной длины; с помощью токена-заполнителя мы дополняем более короткие тексты до наибольшей длины, чтобы все тексты имели одинаковую длину).
    - [UNK] используется для представления слов, которые не включены в словарь.

- Обратите внимание, что `GPT-2` не требует ни одного из упомянутых выше токенов, а использует только токен `<|endoftext|>` для уменьшения сложности.  
`<|endoftext|>` аналогичен упомянутому выше токену [EOS].
- GPT также использует `<|endoftext|>` для паддинга (поскольку при обучении на пакетных данных мы обычно используем маску, мы всё равно не будем обращать внимание на токены-заполнители, поэтому не имеет значения, что это за токены).
- GPT-2 не использует токен <UNK> для слов, отсутствующих в словаре; вместо этого GPT-2 использует токенизатор на основе попарного кодирования байтов (BPE), который разбивает слова на подслова (суб-единицы), что мы обсудим в следующем разделе.

- Біз `<|endoftext|>` токендерін екі тәуелсіз мәтін көзінің арасында қолданамыз:
- Мы используем токены `<|endoftext|>` между двумя независимыми источниками текста:


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="1000px">

- Төмендегі мәтінді `токендегенде` не болатынын көрейік:
- Давайте посмотрим, что произойдёт, если мы `токенизируем` следующий текст:

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

- Жоғарыдағы код қате шығарады, себебі `«Hello»` сөзі сөздікте жоқ.  
- Мұндай жағдайларды шешу үшін сөздікке бейтаныс сөздерді белгілейтін `<|unk|>` сияқты арнайы токендерді қоса аламыз.
- Сөздікті кеңейтіп жатқандықтан, GPT-2 оқытуында мәтіннің соңын белгілеу үшін қолданылатын `<|endoftext|>` деп аталатын тағы бір токен қосайық (ол сондай-ақ біріктірілген мәтіндер арасында да қолданылады, мысалы, егер оқыту деректер жиынымыз бірнеше мақаладан, кітаптан және т.б. тұрса).

- Приведенный выше код выдаёт ошибку, потому что слово `«Hello»` отсутствует в словаре.
- Чтобы справиться с такими случаями, мы можем добавить в словарь специальные токены, такие как `<|unk|>`, для представления неизвестных слов.
- Поскольку мы уже расширяем словарь, давайте добавим ещё один токен, `<|endoftext|>`,  
который используется при обучении GPT-2 для обозначения конца текста (он также используется между объединёнными текстами,    
например, если наши обучающие датасеты состоят из нескольких статей, книг и т. д.).

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
len(vocab.items())

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        # Preprocess the text
        preprocessed = re.split(r'([,.:;?_!"()—*’“$%\']|--|——|\s)', text)
        # Remove empty strings
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Replace unknown tokens
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]
        # Convert tokens to IDs
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        # Convert IDs back to tokens
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?_!"()—*’“$%\'])', r'\1', text)
        return text

- Өзгертілген токендеу құралымен мәтінді токендеп көрейік:
- Давайте попробуем токенизировать текст с помощью изменённого токенизатора:

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

In [ ]:
tokenizer.encode(text)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

## 2.5 BytePair кодтау (Байт жұбымен кодтау)
## 2.5 BytePair кодирование (Попарное кодирование байтов)

- GPT-2 өз токендегіші ретінде BytePair (Байт жұбымен кодтауды) (BPE) пайдаланды.
- Бұл модельге алдын ала анықталған сөздікте жоқ сөздерді кішірек сөз бөліктеріне немесе тіпті жеке таңбаларға бөлуге мүмкіндік береді, осылайша ол сөздіктен тыс сөздерді өңдей алады.
- Мысалы, егер GPT-2 сөздігінде `«unfamiliarword»` деген сөз болмаса, ол оны өзінің үйретілген BPE біріктірулеріне байланысты `["unfam", "iliar", "word"]` немесе басқа сөз бөліктеріне бөліп токендеуі мүмкін.   
**Мысалы:**    
`Walker walked a long way` -> `[w, a, l, k, e, r, d, o, n, g, y, al, alk, alke, walke]`
***
- Түпнұсқа BPE токендегішін мына жерден табуға болады: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- Бұл тарауда біз OpenAI-дың ашық бастапқы кодты `tiktoken` пакеттың `BPE` токендегішін қолданамыз. Бұл пакеттың негізгі алгоритмдері есептеу өнімділігін жақсарту үшін Rust тілінде жазылған.

- GPT-2 использовал попарное кодирование байтов (BPE) в качестве своего токенизатора.
- Это позволяет модели разбивать слова, которых нет в её предопределённом словаре, на более мелкие подслова или даже отдельные символы, что даёт возможность обрабатывать слова, отсутствующие в словаре.
- Например, если в словаре GPT-2 нет слова «unfamiliarword», он может токенизировать его как ["unfam", "iliar", "word"] или разбить на другие подслова, в зависимости от своих обученных слияний BPE.    
**Пример:**    
`Walker walked a long way` -> `[w, a, l, k, e, r, d, o, n, g, y, al, alk, alke, walke]`  
***
- Оригинальный токенизатор BPE можно найти здесь: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- В этой главе мы используем токенизатор BPE из библиотеки с открытым исходным кодом tiktoken от OpenAI, которая реализует свои основные алгоритмы на Rust для повышения вычислительной производительности.

## LINK
* BPE visualizer: https://www.bpe-visualizer.com

In [ ]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

In [ ]:
strings = tokenizer.decode(integers)

print(strings)

- BPE токендегіштері бейтаныс сөздерді сөз бөліктеріне және жеке таңбаларға бөледі:
- BPE-токенизаторы разбивают неизвестные слова на подслова и отдельные символы:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="1000px">

## 2.6 Жылжымалы терезе әдісі арқылы деректерді іріктеу
## 2.6 Выборка данных методом скользящего окна

- Біз АТМ-дерді  бір сөз генерациялауға үйрететіндіктен, оқыту деректерде келесі сөз  ағымдағы сөзге болжау үшін нысана болатындай етіп дайындаймыз:
- Мы обучаем LLM генерировать по одному слову за раз, поэтому обучающие данные нужно подготовить так, чтобы следующее слово в последовательности было целью для предсказания :

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="1000px">

In [ ]:
with open("time_machine.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
# Encoding text using new tokenizer
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

- Әрбір мәтін бөлігі үшін бізге кіріс деректер мен нысана деректер қажет.
- Модельдің келесі сөзді болжағанын қалайтындықтан, нысаналар дегеніміз — оңға қарай бір позицияға ығыстырылған кіріс деректер.

<br>

- Для каждого фрагмента текста нам нужны входящие данные и целевые данные.  
- Поскольку мы хотим, чтобы модель предсказывала следующее слово, цели — это входы, сдвинутые на одну позицию вправо.

In [ ]:
enc_sample = enc_text[50:]

In [ ]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

- Болжау әдісі төмендегідей болады:
- Шаг за шагом предсказание будет выглядеть следующим образом:

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

- Келесі сөзді болжауды біз `зейін механизмін` (attention mechanism) қарастырып болғаннан кейін, кейінгі тарауда талқылаймыз.
- Әзірше біз кіріс деректер жиынын топтастап, бір позицияға ығыстырылған кіріс деректер мен нысана деректерді қайтаратын қарапайым деректер жүктегішін жүзеге асырамыз.

- Мы займёмся предсказанием следующего слова в одной из последующих глав, после того как изучим механизм внимания (attention mechanism).
- А пока мы реализуем простой загрузчик данных (dataloader), который проходит по входному набору данных и возвращает входы и цели, сдвинутые на одну позицию.

In [ ]:
import torch
print("PyTorch version:", torch.__version__)

- Позицияны +1 сөзге өзгерте отырып, жылжымалы терезе әдісін қолданамыз:
- Мы используем подход скользящего окна, смещая позицию на +1 слово:


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="1000px">

- Кіріс мәтіндік деректер жиынынан бөліктерді алатын деректер жиынын (dataset) және деректер жүктегішін (dataloader) құру.
- Создание набора данных (dataset) и загрузчика данных (dataloader), которые извлекают фрагменты из входного текстового набора данных.

In [ ]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            # The input is a chunk of text of size 'max_length'
            input_chunk = token_ids[i:i + max_length]
            # The target is the same chunk, but shifted one position to the right.
            target_chunk = token_ids[i + 1: i + max_length + 1]

            # Convert the chunks to PyTorch tensors and store them.
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    # Returns the total number of samples in the dataset.
    def __len__(self):
        return len(self.input_ids)
        
    # Retrieves one sample (an input and its corresponding target) from the dataset.
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- Контекст өлшемі (сөздер саны) 4 болатын АТМ үшін топтама өлшемі 1-ге тең деректер жүктегішін (dataloader) тексеріп көрейік:
- Давайте протестируем загрузчик данных (dataloder) с размером батча 1 для LLM с размером контекста 4:


In [ ]:
with open("time_machine.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Create a dataloader instance using our custom function.
dataloader = create_dataloader_v1(
                raw_text,           # The source text to be processed into chunks.
                batch_size=1,       # The number of text chunks in each batch.
                max_length=4,       # The fixed length of each text chunk (in tokens).
                stride=1,           # The step size to slide the window for creating chunks.
                shuffle=False       # Whether to shuffle the order of the chunks (False means keep order).
)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

In [ ]:
second_batch = next(data_iter)
print(second_batch)

- Төменде қадамы контекст ұзындығына (бұл жерде: 4) тең болатын мысал көрсетілген:
- Ниже приведён пример, в котором шаг равен длине контекста (здесь: 4):

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="1000px">

- Біз сондай-ақ  нәтижелер деректерін  топтамамен (batch) де шығара аламыз.
- Ескеріңіз, біз бұл жерде топтамалар арасында қиылысулар болмауы үшін қадамды ұлғайтамыз, себебі қиылысудың көп болуы шамадан тыс үйретудің артуына әкелуі мүмкін.

<br>

- Мы также можем создавать пакетные (batch) выходные данные.
- Обратите внимание, что здесь мы увеличиваем шаг, чтобы не было пересечений между пакетами, поскольку большее пересечение может привести к усилению переобучения.

In [ ]:
dataloader = create_dataloader_v1(
                raw_text, 
                batch_size=8, 
                max_length=4, 
                stride=4, 
                shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

## 2.7 Токен эмбеддингтерін құру
## 2.7 Создание эмбеддинги токенов

- Деректер LLM (АТМ) үйрету үшін дайын деуге болады.
- Соңында, эмбеддинг қабатын пайдаланып, токендерді үздіксіз векторлық көрініске (vectors) эмбеддейік.
- Әдетте, бұл эмбеддинг қабаттары LLM-нің өз құрамына кіреді және модельді оқыту барысында жаңартылып (оқытылып) отырады.

<br>

- Данные уже почти готовы для обучения LLM.
- И наконец, давайте преобразуем токены в непрерывное векторное представление с помощью слоя эмбеддингов.
- Обычно эти слои эмбеддингов являются частью самой LLM и обновляются (обучаются) в процессе обучения модели.


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="1000px">

- Бізде (токендеуден кейін) кіріс ID-лары 2, 3, 5 және 1 болатын келесі төрт кіріс мысалы бар деп есептейік.
- Предположим, у нас есть следующие четыре входных примера с ID 2, 3, 5 и 1 (после токенизации).

In [ ]:
input_ids = torch.tensor([2, 3, 5, 1])

- Қарапайымдылық үшін, бізде бар болғаны 6 сөзден тұратын шағын сөздік бар және біз өлшемі 3-ке тең эмбеддингтер жасағымыз келеді деп есептейік:
- Для простоты предположим, что у нас есть небольшой словарь всего из 6 слов, и мы хотим создать эмбеддинги размером 3:

In [ ]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
print(embedding_layer.weight)

- ID-і 3-ке тең токенді 3-өлшемді векторға түрлендіру үшін келесіні орындаймыз:
- Чтобы преобразовать токен с ID 3 в 3-мерный вектор, выполним следующее:

In [ ]:
print(embedding_layer(torch.tensor([0])))

In [ ]:
input_ids

In [ ]:
print(embedding_layer(input_ids))

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="1000px">

## 2.8 Сөз позицияларын кодтау
## 2.8 Кодирование позиций слов

- Эмбеддинг қабаты токендерды деректерде 
орналасу орнын есепке алмай векторлық көріністерге айналдыра береді:
- Слой эмбеддингов преобразует токены в идентичные векторные представления независимо от их расположения во входной последовательности:


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="1000px">

- **Позициялық эмбеддингтер** **ауқымды тілдік модель** үшін **кіріс эмбеддингтерін** құру мақсатында **токен эмбеддинг векторымен** біріктіріледі:

-   **Позиционные эмбеддинги** объединяются с **вектором эмбеддинга токена**, чтобы сформировать **входные эмбеддинги** для **большой языковой модели**:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="1000px">

-   **Байт жұбымен кодтаушының (BytePair encoder)** **сөздік қорының өлшемі 50 257-ге тең**:
-   **Кіріс токендерін** **256-өлшемді векторлық көрініске** кодтағымыз келеді деп есептейік:

<br>

-   **Кодировщик BytePair** имеет **размер словаря 50 257**:
-   Предположим, мы хотим закодировать **входные токены** в **256-мерное векторное представление**:

In [ ]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- Деректер жүктегішінен (dataloader) деректерді іріктесек, біз әрбір топтамадағы токендерді 256-өлшемді векторға эмбеддейміз.
- Егер бізде әрқайсысында 4 токені бар топтама (batch) өлшемі 8 болса, нәтижесінде 8 x 4 x 256 тензоры пайда болады:

<br>

- Если мы делаем выборку данных из загрузчика (dataloader), мы преобразуем токены в каждом пакете в 256-мерный вектор.
- Если у нас размер пакета (батча) равен 8 и в каждом по 4 токена, в результате мы получим тензор размером 8 x 4 x 256:

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(
                raw_text, 
                batch_size=8, 
                max_length=max_length,
                stride=max_length, 
                shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# First group of words
# print(token_embeddings[0])

- GPT-2 абсолютті позициялық эмбеддингтерді пайдаланады, 
сондықтан біз жай ғана тағы бір эмбеддинг қабатын құрамыз:
- GPT-2 использует абсолютные позиционные эмбеддинги, 
поэтому мы просто создаём ещё один слой эмбеддингов:

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# print(pos_embedding_layer.weight)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

print(pos_embeddings)

In [ ]:
print(token_embeddings[0])

- LLM-де қолданылатын кіріс эмбеддингтерін құру үшін біз токендік және позициялық эмбеддингтерді жай ғана қосамыз:
- Чтобы создать входные эмбеддинги, используемые в LLM, мы просто складываем токенные и позиционные эмбеддинги:

In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

print(input_embeddings[0])

- Кіріс деректерді өңдеу процесінің бастапқы кезеңінде кіріс мәтіні жеке токендерге бөлінеді.
- Осы сегменттеуден кейін бұл токендер алдын ала анықталған сөздік негізінде токен ID-ларына айналдырылады:

<br>

- На начальном этапе процесса обработки входных данных входной текст сегментируется на отдельные токены.
- После этой сегментации эти токены преобразуются в идентификаторы токенов (ID) на основе предопределённого словаря:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="1000px">

# Links

* BPE visualizer: https://www.bpe-visualizer.com
* Tokenizer visualizer: https://tiktokenizer.vercel.app